In [ ]:
import os
import pickle
import pandas as pd
import sys
sys.path.append('..')
sys.path.append('../RecStudio')

In [ ]:
dataset = 'amazon-sports'
if dataset == 'amazon-toys':
    dataset_dir = 'Toys_and_Games'
elif dataset == 'amazon-sports':
    dataset_dir = 'Sports_and_Outdoors'
elif dataset == 'movielens':
    dataset_dir = 'MovieLens'
elif dataset == 'steam':
    dataset_dir = 'Steam'

In [ ]:
# Datasets
dataset_dir = os.path.join(os.getenv('DATA_MOUNT_DIR'), dataset_dir)
with open(os.path.join(dataset_dir, 'truncated20_dataset.pkl'), 'rb') as f:
    _ = pickle.load(f)
    _ = pickle.load(f)
    _ = pickle.load(f)
    dataset = pickle.load(f)
dataset.dataframe2tensors()

In [ ]:
field2idx2token = {}
for f, token2idx in dataset.field2token2idx.items():
    field2idx2token[f] = {id: t for t, id in token2idx.items()}

In [ ]:
inter_feat = dataset.inter_feat
corpus = []
if dataset.name == 'amazon':
    for _ in range(len(inter_feat)):
        for k, v in inter_feat[_].items():
            if k == dataset.fuid:
                user_sentence = f'user_id: {v.item()}'
            if k == dataset.fiid:
                feat = dataset.item_feat[v.item()]
                item_sentence = f'item_id: {v.item()}'
                price = dataset.float_field_preprocess['price'].inverse_transform(feat['price'].reshape(-1, 1)).item()
                brand = field2idx2token['brand'][feat['brand'].item()]
                category = []
                for _ in feat['category']:
                    c = field2idx2token['category'][_.item()]
                    if c != '[PAD]':
                        category.append(c)
                desc = feat['description']
                if price != 0:
                    item_sentence += f', price: {price}'
                if brand != '[PAD]':
                    item_sentence += f', brand: {brand}'
                if len(category) > 0:
                    category = ', '.join(category)
                    item_sentence += f', category: {category}'
                if desc != 0:
                    item_sentence += f', description: {desc}'
        sentence = ', '.join([user_sentence, item_sentence]) + '\n'
        corpus.append(sentence)

elif dataset.name == 'movielens':
    for _ in range(len(inter_feat)):
        for k, v in inter_feat[_].items():
            if k == dataset.fuid:
                user_sentence = f'user_id: {v.item()}'
            if k == dataset.fiid:
                feat = dataset.item_feat[v.item()]
                item_sentence = f'item_id: {v.item()}'
                
                genres = []
                for _ in feat['genres']:
                    c = field2idx2token['genres'][_.item()]
                    if c != '[PAD]':
                        genres.append(c)
                desc = feat['summary']
                if len(genres) > 0:
                    genres = ', '.join(genres)
                    item_sentence += f', genres: {genres}'
                if desc != 0:
                    item_sentence += f', summary: {desc}'
        sentence = ', '.join([user_sentence, item_sentence]) + '\n'
        corpus.append(sentence)

In [ ]:
with open(os.path.join(os.getenv('DATA_MOUNT_DIR'), dataset_dir, 'corpus.txt'), 'w') as f:
    for _ in corpus:
        f.write(_) 